In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## やったこと

- テキストのクリーニング処理 -> 改善
- host, categoryのカテゴリカル変数をエンベディングして入力 -> 改善

- epochs=20で、early-stoppingはあまり良くならなかった -> とりあえず速く数を回したいので、epochs=4でやっている


- batch_size=8以上にすると、out_of_memoryになる

- MSELossを使用 -> 悪化
- titleは分けて、別のエンベディングとして入力 -> 悪化



- BERTを2つ使う -> gpu不足
- クラス分類問題にする（30*num_class） -> 学習が安定しない（nan）
- 30個の目的変数それぞれ独立に予測するモデル -> 約30時間必要、あまり精度が出ないように見える -> 関連する目的変数だけをグルーピングしてモデルを分ける必要？

In [0]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os, sys, gc, random, multiprocessing, glob, time

DATA_DIR = '/content/drive/My Drive/Colab Notebooks/GoogleQuest/input/google-quest-challenge'
# DATA_DIR = '../input/google-quest-challenge'
# DATA_DIR = 'D:/project/ICF_AutoCapsule_disabled/kaggle/google-quest-challenge'
# BERT_DIR = 'D:/project/ICF_AutoCapsule_disabled/BERT'

In [0]:
# !pip install ../input/sacremoses/sacremoses-master/
# !pip install ../input/transformers/transformers-master/

In [0]:
!pip install transformers
!pip install flashtext

In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

#from ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

from scipy.stats import spearmanr

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)

from tqdm import tqdm
print(transformers.__version__)

2.4.1


In [0]:
## Make results reproducible .Else noone will believe you .
import random

def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [0]:
class PipeLineConfig:
    def __init__(self, lr, warmup, epochs, patience, batch_size, seed, name, question_weight,answer_weight,fold,train):
        self.lr = lr
        self.warmup = warmup
        self.epochs = epochs
        self.patience = patience
        self.batch_size = batch_size
        self.seed = seed
        self.name = name
        self.question_weight = question_weight
        self.answer_weight =answer_weight
        self.fold = fold
        self.train = train

In [0]:
config = PipeLineConfig(lr=1e-5, \
                        warmup=0.01, \
                        epochs=4, \
                        patience=3, \
                        batch_size=4, \
                        seed=42, \
                        name='twoBERT', \
                        question_weight=0.5, \
                        answer_weight=0.5, \
                        fold=5, \
                        train=True
                       )

In [0]:
seed_everything(config.seed)

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


In [0]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [0]:
target_columns = sub.columns.values[1:].tolist()
target_columns

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [0]:
train = pd.read_csv(f'{DATA_DIR}/train.csv')
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [0]:
test = pd.read_csv(f'{DATA_DIR}/test.csv')
test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


## Preprocessing

In [0]:
import re
from flashtext import KeywordProcessor

In [0]:
PUNCTS = {
            '》', '〞', '¢', '‹', '╦', '║', '♪', 'Ø', '╩', '\\', '★', '＋', 'ï', '<', '?', '％', '+', '„', 'α', '*', '〰', '｟', '¹', '●', '〗', ']', '▾', '■', '〙', '↓', '´', '【', 'ᴵ',
            '"', '）', '｀', '│', '¤', '²', '‡', '¿', '–', '」', '╔', '〾', '%', '¾', '←', '〔', '＿', '’', '-', ':', '‧', '｛', 'β', '（', '─', 'à', 'â', '､', '•', '；', '☆', '／', 'π',
            'é', '╗', '＾', '▪', ',', '►', '/', '〚', '¶', '♦', '™', '}', '″', '＂', '『', '▬', '±', '«', '“', '÷', '×', '^', '!', '╣', '▲', '・', '░', '′', '〝', '‛', '√', ';', '】', '▼',
            '.', '~', '`', '。', 'ə', '］', '，', '{', '～', '！', '†', '‘', '﹏', '═', '｣', '〕', '〜', '＼', '▒', '＄', '♥', '〛', '≤', '∞', '_', '[', '＆', '→', '»', '－', '＝', '§', '⋅', 
            '▓', '&', 'Â', '＞', '〃', '|', '¦', '—', '╚', '〖', '―', '¸', '³', '®', '｠', '¨', '‟', '＊', '£', '#', 'Ã', "'", '▀', '·', '？', '、', '█', '”', '＃', '⊕', '=', '〟', '½', '』',
            '［', '$', ')', 'θ', '@', '›', '＠', '｝', '¬', '…', '¼', '：', '¥', '❤', '€', '−', '＜', '(', '〘', '▄', '＇', '>', '₤', '₹', '∅', 'è', '〿', '「', '©', '｢', '∙', '°', '｜', '¡', 
            '↑', 'º', '¯', '♫', '#'
          }


mispell_dict = {"aren't" : "are not", "can't" : "cannot", "couldn't" : "could not",
"couldnt" : "could not", "didn't" : "did not", "doesn't" : "does not",
"doesnt" : "does not", "don't" : "do not", "hadn't" : "had not", "hasn't" : "has not",
"haven't" : "have not", "havent" : "have not", "he'd" : "he would", "he'll" : "he will", "he's" : "he is", "i'd" : "I would",
"i'd" : "I had", "i'll" : "I will", "i'm" : "I am", "isn't" : "is not", "it's" : "it is",
"it'll":"it will", "i've" : "I have", "let's" : "let us", "mightn't" : "might not", "mustn't" : "must not", 
"shan't" : "shall not", "she'd" : "she would", "she'll" : "she will", "she's" : "she is", "shouldn't" : "should not", "shouldnt" : "should not",
"that's" : "that is", "thats" : "that is", "there's" : "there is", "theres" : "there is", "they'd" : "they would", "they'll" : "they will",
"they're" : "they are", "theyre":  "they are", "they've" : "they have", "we'd" : "we would", "we're" : "we are", "weren't" : "were not",
"we've" : "we have", "what'll" : "what will", "what're" : "what are", "what's" : "what is", "what've" : "what have", "where's" : "where is",
"who'd" : "who would", "who'll" : "who will", "who're" : "who are", "who's" : "who is", "who've" : "who have", "won't" : "will not", "wouldn't" : "would not", "you'd" : "you would",
"you'll" : "you will", "you're" : "you are", "you've" : "you have", "'re": " are", "wasn't": "was not", "we'll":" will", "didn't": "did not", "tryin'":"trying"}


kp = KeywordProcessor(case_sensitive=True)
for k, v in mispell_dict.items():
    kp.add_keyword(k, v)

def clean_punct(text):
    text = str(text)
    for punct in PUNCTS:
        text = text.replace(punct, ' {} '.format(punct))
    return text


def preprocessing(text):
    text = text.lower()
    text = re.sub(r'(\&lt)|(\&gt)', ' ', text)
    
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', text)
    text = kp.replace_keywords(text)
    text = clean_punct(text)
    text = re.sub(r'\n\r', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text

## Dataset

In [0]:
MAX_LEN = 512

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        #self.tokenizer = BertTokenizer.from_pretrained(BERT_DIR+'/bert-base-uncased')
        #self.tokenizer = BertTokenizer.from_pretrained('../input/bert-base-uncased/')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]

        question_title = row.question_title
        question_body = row.question_body
        answer_text = row.answer


        inputs_q = self.tokenizer.encode_plus(
            question_title + " " + question_body,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )

        inputs_a = self.tokenizer.encode_plus(
            answer_text,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )

        ids_q = inputs_q["input_ids"]
        token_type_ids_q = inputs_q["token_type_ids"]
        mask_q = inputs_q["attention_mask"]

        ids_a = inputs_a["input_ids"]
        token_type_ids_a = inputs_a["token_type_ids"]
        mask_a = inputs_a["attention_mask"]
        
        padding_length_q = MAX_LEN - len(ids_q)
        padding_length_a = MAX_LEN - len(ids_a)
        
        ids_q = ids_q + ([0] * padding_length_q)
        mask_q = mask_q + ([0] * padding_length_q)
        token_type_ids_q = token_type_ids_q + ([0] * padding_length_q)

        ids_a = ids_a + ([0] * padding_length_a)
        mask_a = mask_a + ([0] * padding_length_a)
        token_type_ids_a = token_type_ids_a + ([0] * padding_length_a)
        
        if self.labeled:
            labels = self.get_label(row)
            return {
                'ids_q': torch.tensor(ids_q, dtype=torch.long),
                'mask_q': torch.tensor(mask_q, dtype=torch.long),
                'token_type_ids_q': torch.tensor(token_type_ids_q, dtype=torch.long),
                'ids_a': torch.tensor(ids_a, dtype=torch.long),
                'mask_a': torch.tensor(mask_a, dtype=torch.long),
                'token_type_ids_a': torch.tensor(token_type_ids_a, dtype=torch.long),
                'labels': labels, 
            }
        else:
            return {
                'ids_q': torch.tensor(ids_q, dtype=torch.long),
                'mask_q': torch.tensor(mask_q, dtype=torch.long),
                'token_type_ids_q': torch.tensor(token_type_ids_q, dtype=torch.long),
                'ids_a': torch.tensor(ids_a, dtype=torch.long),
                'mask_a': torch.tensor(mask_a, dtype=torch.long),
                'token_type_ids_a': torch.tensor(token_type_ids_a, dtype=torch.long)
            }


    def get_label(self, row):
        return torch.tensor(row[target_columns].values.astype(np.float32))

In [0]:
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0):
    df = pd.read_csv(f'{DATA_DIR}/train.csv')

    # cleaning
    df['question_title'] = df['question_title'].apply(lambda x : preprocessing(x))
    df['question_body'] = df['question_body'].apply(lambda x : preprocessing(x))
    df['answer'] = df['answer'].apply(lambda x : preprocessing(x))
    

    df = shuffle(df, random_state=1234)
    gkf = GroupKFold(n_splits=5).split(X=df.question_body, groups=df.question_body)
    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold == ifold:
            df_train = df.iloc[train_idx]
            df_val = df.iloc[valid_idx]
            break

    print('train', df_train.shape)
    print('val', df_val.shape)

    ds_train = QuestDataset(df_train, train_mode=True)
    train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
    train_loader.num = len(df_train)

    ds_val = QuestDataset(df_val, train_mode=False)
    val_loader = torch.utils.data.DataLoader(ds_val, batch_size=val_batch_size, shuffle=False, num_workers=0, drop_last=False)
    val_loader.num = len(df_val)
    val_loader.df = df_val

    return train_loader, val_loader, df_val.shape[0], valid_idx


def get_test_loader(batch_size=4):
    df = pd.read_csv(f'{DATA_DIR}/test.csv')

    # cleaning
    df['question_title'] = df['question_title'].apply(lambda x : preprocessing(x))
    df['question_body'] = df['question_body'].apply(lambda x : preprocessing(x))
    df['answer'] = df['answer'].apply(lambda x : preprocessing(x))
    
    ds_test = QuestDataset(df, train_mode=False, labeled=False)
    loader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=False)
    loader.num = len(df)
    
    return loader

In [0]:
# class QuestModel(nn.Module):
#     def __init__(self, n_classes=30):
#         super(QuestModel, self).__init__()
#         self.model_name = 'QuestModel'
#         #self.bert_model = BertModel.from_pretrained(BERT_DIR+'/bert-base-uncased/')
#         #self.bert_model = BertModel.from_pretrained('../input/bert-base-uncased/')
#         self.bert_model_q = BertModel.from_pretrained('bert-base-uncased')
#         self.bert_model_a = BertModel.from_pretrained('bert-base-uncased')

#         # self.lstm_q = nn.LSTM(768, 300)
#         # self.lstm_a = nn.LSTM(768, 300)
        
#         self.fc_q1 = nn.Linear(768*2, 12)
#         self.fc_q2 = nn.Linear(768*2, 9)
        
#         self.fc_a = nn.Linear(768*2+21, 9)

#     def forward(self, ids_q, mask_q, token_type_ids_q, ids_a, mask_a, token_type_ids_a):
#         layers_q, pool_out_q = self.bert_model_q(input_ids=ids_q, token_type_ids=token_type_ids_q, attention_mask=mask_q)
#         layers_a, pool_out_a = self.bert_model_a(input_ids=ids_a, token_type_ids=token_type_ids_a, attention_mask=mask_a)
        
#         # #print(layers_q.shape)
#         # layers_q, _ = self.lstm_q(layers_q)
#         # layers_a, _ = self.lstm_a(layers_a)

#         #print(layers_q.shape)
#         out_q = F.avg_pool1d(layers_q.transpose(1,2), kernel_size=layers_q.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
#         out_a = F.avg_pool1d(layers_a.transpose(1,2), kernel_size=layers_a.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
#         out = torch.cat([out_q, out_a], dim=-1)
#         out = F.dropout(out, p=0.2, training=self.training)

#         logit_q1 = self.fc_q1(out)
#         logit_q2 = self.fc_q2(out)
        
#         out_a = torch.cat([out, F.relu(logit_q1)], dim=-1)
#         out_a = torch.cat([out_a, F.relu(logit_q2)], dim=-1)
#         logit_a = self.fc_a(out_a)

#         logit = torch.cat([logit_q1, logit_q2], dim=-1)
#         logit = torch.cat([logit, logit_a], dim=-1)

#         logit = logit[:, [0,1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,12,20,21,22,23,24,25,26,27,28,29]]

#         return logit
    

In [0]:
class BertForSequenceClassification_v2(nn.Module):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]
    """
    def __init__(self):

        super(BertForSequenceClassification_v2, self).__init__()

       # config.output_hidden_states=True (make sure)
        self.num_labels = 30
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(768, 30)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
            position_ids=None, head_mask=None, inputs_embeds=None, labels=None, 
            extra_feats=None):

        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds)

        # sequence_output = outputs[0]
        # pooled_output = outputs[1]

        hidden_states = outputs[2] #hidden_states: 12 layers tuples each is of (batch_size, sequence_length, hidden_size) + embedding``
        # print(seq[-1].shape, seq[-1][:, 0].shape)

        # we are taking zero because in my understanding that's the [CLS] token...
        # idea is to pool last 4 layers as well instead of just the last one, since it's too close to the output
        # layers, it might not be that efficient as it's more regulated by the o/p's..

        h12 = hidden_states[-1][:, 0].reshape((-1, 1, 768))
        h11 = hidden_states[-2][:, 0].reshape((-1, 1, 768))
        h10 = hidden_states[-3][:, 0].reshape((-1, 1, 768))
        h9  = hidden_states[-4][:, 0].reshape((-1, 1, 768))

        all_h = torch.cat([h9, h10, h11, h12], 1) #Also don't forget to add the last CLS token seq_op/pooled_op as you wish..
        mean_pool = torch.mean(all_h, 1)

        pooled_output = self.dropout(mean_pool)
        logits = self.classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [0]:
class QuestModel(nn.Module):
    def __init__(self, n_classes=30):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        #self.bert_model = BertModel.from_pretrained(BERT_DIR+'/bert-base-uncased/')
        #self.bert_model = BertModel.from_pretrained('../input/bert-base-uncased/')
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        
        self.fc_q = nn.Linear(768*20, 21)
        self.fc_a = nn.Linear(768*20+21, 9)

    def forward(self, ids_q, mask_q, token_type_ids_q, ids_a, mask_a, token_type_ids_a):
        layers_q, pool_out_q = self.bert_model(input_ids=ids_q, token_type_ids=token_type_ids_q, attention_mask=mask_q)
        layers_a, pool_out_a = self.bert_model(input_ids=ids_a, token_type_ids=token_type_ids_a, attention_mask=mask_a)
        

        # [batch_size, 512, 768]
        layers_q1,layers_q2,layers_q3,layers_q4,layers_q5,layers_q6,layers_q7,layers_q8,layers_q9,layers_q10,layers_q11,layers_q12,layers_q13,layers_q14,layers_q15,layers_q16,layers_q17,layers_q18,layers_q19,layers_q20 = layers_q[:, :25, :], layers_q[:, 25:50, :], layers_q[:,50:75, :], layers_q[:, 75:100, :], layers_q[:, 125:150, :], layers_q[:, 150:175, :], layers_q[:, 175:200, :], layers_q[:, 200:225, :], layers_q[:, 225:250, :], layers_q[:, 250:275, :], layers_q[:, 275:300, :], layers_q[:, 300:325, :], layers_q[:, 325:350, :], layers_q[:, 350:375, :], layers_q[:, 375:400, :], layers_q[:, 400:425, :], layers_q[:, 425:450, :], layers_q[:, 450:475, :], layers_q[:, 475:500, :], layers_q[:, 500:512, :]
        layers_a1,layers_a2,layers_a3,layers_a4,layers_a5,layers_a6,layers_a7,layers_a8,layers_a9,layers_a10,layers_a11,layers_a12,layers_a13,layers_a14,layers_a15,layers_a16,layers_a17,layers_a18,layers_a19,layers_a20 = layers_a[:, :25, :], layers_a[:, 25:50, :], layers_a[:,50:75, :], layers_a[:, 75:100, :], layers_a[:, 125:150, :], layers_a[:, 150:175, :], layers_a[:, 175:200, :], layers_a[:, 200:225, :], layers_a[:, 225:250, :], layers_a[:, 250:275, :], layers_a[:, 275:300, :], layers_a[:, 300:325, :], layers_a[:, 325:350, :], layers_a[:, 350:375, :], layers_a[:, 375:400, :], layers_a[:, 400:425, :], layers_a[:, 425:450, :], layers_a[:, 450:475, :], layers_a[:, 475:500, :], layers_a[:, 500:512, :]
        out_q1 = F.avg_pool1d(layers_q1.transpose(1,2), kernel_size=layers_q1.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a1 = F.avg_pool1d(layers_a1.transpose(1,2), kernel_size=layers_a1.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q2 = F.avg_pool1d(layers_q2.transpose(1,2), kernel_size=layers_q2.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a2 = F.avg_pool1d(layers_a2.transpose(1,2), kernel_size=layers_a2.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q3 = F.avg_pool1d(layers_q3.transpose(1,2), kernel_size=layers_q3.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a3 = F.avg_pool1d(layers_a3.transpose(1,2), kernel_size=layers_a3.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q4 = F.avg_pool1d(layers_q4.transpose(1,2), kernel_size=layers_q4.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a4 = F.avg_pool1d(layers_a4.transpose(1,2), kernel_size=layers_a4.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q5 = F.avg_pool1d(layers_q5.transpose(1,2), kernel_size=layers_q5.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a5 = F.avg_pool1d(layers_a5.transpose(1,2), kernel_size=layers_a5.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q6 = F.avg_pool1d(layers_q6.transpose(1,2), kernel_size=layers_q6.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a6 = F.avg_pool1d(layers_a6.transpose(1,2), kernel_size=layers_a6.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q7 = F.avg_pool1d(layers_q7.transpose(1,2), kernel_size=layers_q7.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a7 = F.avg_pool1d(layers_a7.transpose(1,2), kernel_size=layers_a7.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q8 = F.avg_pool1d(layers_q8.transpose(1,2), kernel_size=layers_q8.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a8 = F.avg_pool1d(layers_a8.transpose(1,2), kernel_size=layers_a8.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q9 = F.avg_pool1d(layers_q9.transpose(1,2), kernel_size=layers_q9.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a9 = F.avg_pool1d(layers_a9.transpose(1,2), kernel_size=layers_a9.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q10 = F.avg_pool1d(layers_q10.transpose(1,2), kernel_size=layers_q10.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a10 = F.avg_pool1d(layers_a10.transpose(1,2), kernel_size=layers_a10.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q11 = F.avg_pool1d(layers_q11.transpose(1,2), kernel_size=layers_q11.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a11 = F.avg_pool1d(layers_a11.transpose(1,2), kernel_size=layers_a11.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q12 = F.avg_pool1d(layers_q12.transpose(1,2), kernel_size=layers_q12.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a12 = F.avg_pool1d(layers_a12.transpose(1,2), kernel_size=layers_a12.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q13 = F.avg_pool1d(layers_q13.transpose(1,2), kernel_size=layers_q13.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a13 = F.avg_pool1d(layers_a13.transpose(1,2), kernel_size=layers_a13.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q14 = F.avg_pool1d(layers_q14.transpose(1,2), kernel_size=layers_q14.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a14 = F.avg_pool1d(layers_a14.transpose(1,2), kernel_size=layers_a14.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q15 = F.avg_pool1d(layers_q15.transpose(1,2), kernel_size=layers_q15.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a15 = F.avg_pool1d(layers_a15.transpose(1,2), kernel_size=layers_a15.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q16 = F.avg_pool1d(layers_q16.transpose(1,2), kernel_size=layers_q16.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a16 = F.avg_pool1d(layers_a16.transpose(1,2), kernel_size=layers_a16.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q17 = F.avg_pool1d(layers_q17.transpose(1,2), kernel_size=layers_q17.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a17 = F.avg_pool1d(layers_a17.transpose(1,2), kernel_size=layers_a17.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q18 = F.avg_pool1d(layers_q18.transpose(1,2), kernel_size=layers_q18.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a18 = F.avg_pool1d(layers_a18.transpose(1,2), kernel_size=layers_a18.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q19 = F.avg_pool1d(layers_q19.transpose(1,2), kernel_size=layers_q19.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a19 = F.avg_pool1d(layers_a19.transpose(1,2), kernel_size=layers_a19.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_q20 = F.avg_pool1d(layers_q20.transpose(1,2), kernel_size=layers_q20.size()[1]).squeeze()  # sequence方向は中央値だけ抽出
        out_a20 = F.avg_pool1d(layers_a20.transpose(1,2), kernel_size=layers_a20.size()[1]).squeeze()  # sequence方向は中央値だけ抽出

        out_q = torch.cat([out_q1, out_q2], dim=-1)
        out_q = torch.cat([out_q, out_q3], dim=-1)
        out_q = torch.cat([out_q, out_q4], dim=-1)
        out_q = torch.cat([out_q, out_q5], dim=-1)
        out_q = torch.cat([out_q, out_q6], dim=-1)
        out_q = torch.cat([out_q, out_q7], dim=-1)
        out_q = torch.cat([out_q, out_q8], dim=-1)
        out_q = torch.cat([out_q, out_q9], dim=-1)
        out_q = torch.cat([out_q, out_q10], dim=-1)
        out_q = torch.cat([out_q, out_q11], dim=-1)
        out_q = torch.cat([out_q, out_q12], dim=-1)
        out_q = torch.cat([out_q, out_q13], dim=-1)
        out_q = torch.cat([out_q, out_q14], dim=-1)
        out_q = torch.cat([out_q, out_q15], dim=-1)
        out_q = torch.cat([out_q, out_q16], dim=-1)
        out_q = torch.cat([out_q, out_q17], dim=-1)
        out_q = torch.cat([out_q, out_q18], dim=-1)
        out_q = torch.cat([out_q, out_q19], dim=-1)
        out_q = torch.cat([out_q, out_q20], dim=-1)
        out_q = F.dropout(out_q, p=0.4, training=self.training)
        logit_q = self.fc_q(out_q)
        

        out_a = torch.cat([out_a1, out_a2], dim=-1)
        out_a = torch.cat([out_a, out_a3], dim=-1)
        out_a = torch.cat([out_a, out_a4], dim=-1)
        out_a = torch.cat([out_a, out_a5], dim=-1)
        out_a = torch.cat([out_a, out_a6], dim=-1)
        out_a = torch.cat([out_a, out_a7], dim=-1)
        out_a = torch.cat([out_a, out_a8], dim=-1)
        out_a = torch.cat([out_a, out_a9], dim=-1)
        out_a = torch.cat([out_a, out_a10], dim=-1)
        out_a = torch.cat([out_a, out_a11], dim=-1)
        out_a = torch.cat([out_a, out_a12], dim=-1)
        out_a = torch.cat([out_a, out_a13], dim=-1)
        out_a = torch.cat([out_a, out_a14], dim=-1)
        out_a = torch.cat([out_a, out_a15], dim=-1)
        out_a = torch.cat([out_a, out_a16], dim=-1)
        out_a = torch.cat([out_a, out_a17], dim=-1)
        out_a = torch.cat([out_a, out_a18], dim=-1)
        out_a = torch.cat([out_a, out_a19], dim=-1)
        out_a = torch.cat([out_a, out_a20], dim=-1)
        out_a = F.dropout(out_a, p=0.4, training=self.training)
        out_a = torch.cat([out_a, F.relu(logit_q)], dim=-1)
        logit_a = self.fc_a(out_a)

        logit = torch.cat([logit_q, logit_a], dim=-1)

        return logit

In [0]:
def train_model(train_loader, optimizer, criterion, scheduler):
    model.train()
    avg_loss = 0.    
    for idx, batch in enumerate(tqdm(train_loader)):
        ids_q = batch['ids_q'].to(device)
        mask_q = batch['mask_q'].to(device)
        token_type_ids_q = batch['token_type_ids_q'].to(device)
        ids_a = batch['ids_a'].to(device)
        mask_a = batch['mask_a'].to(device)
        token_type_ids_a = batch['token_type_ids_a'].to(device)
        labels = batch['labels'].to(device)
        
        logits = model(ids_q, mask_q, token_type_ids_q, ids_a, mask_a, token_type_ids_a)        
        loss = criterion(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        avg_loss += loss.item() / len(train_loader)
        del ids_q, mask_q, token_type_ids_q, ids_a, mask_a, token_type_ids_a, labels

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

def val_model(val_loader, val_length, batch_size=8):
    model.eval() # eval mode  
    avg_val_loss = 0.
    
    valid_preds = np.zeros((val_length, len(target_columns)))
    original = np.zeros((val_length, len(target_columns)))
    
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(val_loader)):
            ids_q = batch['ids_q'].to(device)
            mask_q = batch['mask_q'].to(device)
            token_type_ids_q = batch['token_type_ids_q'].to(device)
            ids_a = batch['ids_a'].to(device)
            mask_a = batch['mask_a'].to(device)
            token_type_ids_a = batch['token_type_ids_a'].to(device)
            labels = batch['labels'].to(device)

            logits = torch.sigmoid(model(ids_q, mask_q, token_type_ids_q, ids_a, mask_a, token_type_ids_a))
            
            avg_val_loss += criterion(logits, labels).item() / len(val_loader)
            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
        
        rho_val = np.mean([spearmanr(original[:, i], valid_preds[:,i]).correlation for i in range(valid_preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        score = 0
        for i in range(len(target_columns)):
            print(i, spearmanr(original[:,i], valid_preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], valid_preds[:, i]).correlation)
    
    return avg_val_loss, score/len(target_columns)

In [0]:
cv_list = []
for fold in range(config.fold):
    print('---%d-Fold---'%(fold+1))
    
    patience = 0
    best_loss   = 100.0
    best_score      = -1.
    best_preds = 0
    best_param_loss = None
    best_param_score = None

    model = QuestModel(n_classes=30).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, eps=4e-5)
    criterion = nn.BCEWithLogitsLoss()
    
    for epoch in range(config.epochs):
        
        torch.cuda.empty_cache()
        start_time   = time.time()
        
        train_loader, val_loader, val_length, val_idx = get_train_val_loaders(batch_size=config.batch_size, val_batch_size=config.batch_size, ifold=fold)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps=config.epochs*len(train_loader))
        #scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps=config.epochs*len(train_loader))

        loss_train = train_model(train_loader, optimizer, criterion, scheduler)
        loss_val, score_val = val_model(val_loader, val_length, batch_size=config.batch_size)
        print(f'Epoch {(epoch+1)}, train_loss: {loss_train}, val_loss: {loss_val}, score_val: {score_val}, time: {(time.time()-start_time)}')

        if score_val > best_score:
            best_score = score_val
            best_param_score = model.state_dict()
            print('best_param_score_{}_{}.pt'.format(config.name ,fold+1))
            torch.save(best_param_score, '/content/drive/My Drive/Colab Notebooks/GoogleQuest/best_param_score_{}_{}.pt'.format(config.name ,fold+1))
        else:
            patience += 1
            if patience >= config.patience:
                del train_loader, val_loader, loss_train, loss_val, score_val
                torch.cuda.empty_cache()
                gc.collect()
                break
    
        del train_loader, val_loader, loss_train, loss_val, score_val
        torch.cuda.empty_cache()
        gc.collect()
        
    model.load_state_dict(best_param_score)
    print('best_param_score_{}_{}.pt'.format(config.name ,fold+1))
    torch.save(best_param_score, '/content/drive/My Drive/Colab Notebooks/GoogleQuest/best_param_score_{}_{}.pt'.format(config.name ,fold+1))   
    cv_list.append(best_score)

    torch.cuda.empty_cache()
    gc.collect()
    
print('CV_score: ', np.mean(cv_list))

---1-Fold---
train (4863, 41)
val (1216, 41)


100%|██████████| 304/304 [01:45<00:00,  2.89it/s]


 val_spearman-rho: 0.38255                                                                                                    
0 SpearmanrResult(correlation=0.3799048325364928, pvalue=4.882204998326703e-43)
1 SpearmanrResult(correlation=0.6146067852446458, pvalue=3.2423200731762917e-127)
2 SpearmanrResult(correlation=0.3971302872298802, pvalue=3.2611299192989267e-47)
3 SpearmanrResult(correlation=0.26174830153552464, pvalue=1.6891982243085107e-20)
4 SpearmanrResult(correlation=0.36986941798372375, pvalue=1.0187617586705217e-40)
5 SpearmanrResult(correlation=0.38578860869164144, pvalue=1.9515920400596954e-44)
6 SpearmanrResult(correlation=0.3165133599144484, pvalue=1.067740921666038e-29)
7 SpearmanrResult(correlation=0.4292543253282772, pvalue=1.0904632661742551e-55)
8 SpearmanrResult(correlation=0.5371094909264099, pvalue=8.015522084530773e-92)
9 SpearmanrResult(correlation=0.08498262550999389, pvalue=0.00301932005198514)
10 SpearmanrResult(correlation=0.4497550549378904, pvalue=1.3450

 75%|███████▌  | 912/1215 [15:03<04:59,  1.01it/s]